## Development of resource use functions

In [1]:
import pandas as pd
import _utilisation_result_calculation
import _processing_functions

run_results = pd.read_csv("../data/run_results.csv")
run_results = _processing_functions.make_callsign_column(run_results)

run_results.head()


2025-02-14 11:53:21.593 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-14 11:53:21.594 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


,P_ID,run_number,time_type,event_type,timestamp,timestamp_dt,day,hour,weekday,month,qtr,callsign_group,vehicle_type,ampds_card,age,sex,hems_result,outcome,callsign
0,1,1,71,resource_preferred_resource_group,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,NaN,NaN,17,67.0,Male,NaN,NaN,NaN
1,1,1,car,resource_preferred_vehicle_type,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,NaN,NaN,17,67.0,Male,NaN,NaN,NaN
2,1,1,Preferred resource available and allocated,resource_preferred_outcome,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,71.0,car,17,67.0,Male,NaN,NaN,C71
3,1,1,CC71,resource_use,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,71.0,car,17,67.0,Male,NaN,NaN,C71
4,1,1,arrival,arrival_departure,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,71.0,car,17,67.0,Male,NaN,NaN,C71


In [2]:
params = pd.read_csv("../data/run_params_used.csv")
params

,parameter,value
0,sim_duration,525600.0
1,warm_up_duration,0.0
2,sim_start_date,2025-02-14 08:00:00
3,sim_end_date,2026-02-14 08:00:00
4,warm_up_end_date,2025-02-14 08:00:00
5,amb_data,False
6,model_exec_time,2025-02-14 11:51:12.838309
7,summer_start_date,2025-04-01
8,winter_start_date,2025-10-01


In [3]:
params[params["parameter"] == "sim_duration"]['value'].values[0]

'525600.0'

In [4]:
resource_use_only = run_results[run_results["event_type"].isin(["resource_use", "resource_use_end"])]
resource_use_only

,P_ID,run_number,time_type,event_type,timestamp,timestamp_dt,day,hour,weekday,month,qtr,callsign_group,vehicle_type,ampds_card,age,sex,hems_result,outcome,callsign
3,1,1,CC71,resource_use,240.0,2025-02-14 12:00:00,Fri,12,weekday,2,1,71.0,car,17,67.0,Male,NaN,NaN,C71
10,1,1,CC71,resource_use_end,418.0,2025-02-14 14:58:00,Fri,12,weekday,2,1,71.0,car,17,67.0,Male,Patient Treated (not conveyed),Conveyed by land without DAA,C71
16,2,1,CC71,resource_use,420.0,2025-02-14 15:00:00,Fri,15,weekday,2,1,71.0,car,29,57.0,Female,NaN,NaN,C71
24,2,1,CC71,resource_use_end,519.0,2025-02-14 16:39:00,Fri,15,weekday,2,1,71.0,car,29,57.0,Female,Patient Conveyed,Airlifted,C71
30,3,1,CC70,resource_use,780.0,2025-02-14 21:00:00,Fri,21,weekday,2,1,70.0,car,6,76.0,Male,NaN,NaN,C70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627290,1581,9,CC70,resource_use_end,524287.0,2026-02-13 10:07:00,Fri,9,weekday,2,1,70.0,car,17,41.0,Female,Patient Treated (not conveyed),Conveyed by land without DAA,C70
627298,1582,9,H70,resource_use_end,524374.0,2026-02-13 11:34:00,Fri,10,weekday,2,1,70.0,helicopter,9,80.0,Male,Patient Conveyed,Airlifted,H70
627304,1583,9,H70,resource_use,524520.0,2026-02-13 14:00:00,Fri,14,weekday,2,1,70.0,helicopter,12,70.0,Male,NaN,NaN,H70
627310,1583,9,H70,resource_use_end,524560.0,2026-02-13 14:40:00,Fri,14,weekday,2,1,70.0,helicopter,12,70.0,Male,Stand Down En Route,Unknown,H70


In [5]:
resource_use_wide = resource_use_only[["P_ID", "run_number", "event_type", "timestamp_dt", "callsign_group", "vehicle_type", "callsign"]].pivot(index=["P_ID", "run_number", "callsign_group", "vehicle_type", "callsign"], columns="event_type", values="timestamp_dt").reset_index()

resource_use_wide

event_type,P_ID,run_number,callsign_group,vehicle_type,callsign,resource_use,resource_use_end
0,1,1,71.0,car,C71,2025-02-14 12:00:00,2025-02-14 14:58:00
1,1,2,70.0,car,C70,2025-02-14 09:00:00,2025-02-14 10:12:00
2,1,3,71.0,car,C71,2025-02-14 10:00:00,2025-02-14 11:06:00
3,1,4,70.0,car,C70,2025-02-14 10:00:00,2025-02-14 11:27:00
4,1,5,70.0,car,C70,2025-02-14 22:00:00,2025-02-14 23:03:00
...,...,...,...,...,...,...,...
45140,1763,2,72.0,car,C72,2026-02-12 10:00:00,2026-02-12 10:44:00
45141,1764,2,71.0,helicopter,H71,2026-02-12 15:00:00,2026-02-12 15:41:00
45142,1765,2,70.0,helicopter,H70,2026-02-13 01:00:00,2026-02-13 02:03:00
45143,1766,2,71.0,helicopter,H71,2026-02-13 14:00:00,2026-02-13 14:43:00


In [6]:
resource_use_wide['resource_use'].isna().sum()

0

In [7]:
resource_use_wide['resource_use_end'].isna().sum()

5

In [8]:
type(resource_use_wide['resource_use_end'][0])

str

In [9]:
import _vehicle_calculation

daily_availability, total_avail_hours, total_avail_minutes = _vehicle_calculation.calculate_available_hours(
        params, rota_path="../data/hems_rota_used.csv"
        )

daily_availability

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_day[callsign] = max(0, first_day[callsign] - warm_up_end.hour)
c:\DAAT\DAA_DES\app\_vehicle_calculation.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_day[callsign] = min(last_day[callsign], sim_end.hour)


,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [10]:
total_avail_hours

,callsign,total_available_hours_in_sim,callsign_group
0,H70,6935,70
1,C70,6935,70
2,H71,4016,71
3,C71,3650,71
4,C72,3650,72


In [11]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group
0,H70,416100,70
1,C70,416100,70
2,H71,240960,71
3,C71,219000,71
4,C72,219000,72


In [12]:
resource_use_wide["resource_use_duration"] = _processing_functions.calculate_time_difference(
        resource_use_wide, 'resource_use', 'resource_use_end', unit='minutes'
        )

utilisation_df_per_run = (
        resource_use_wide.groupby(['run_number', 'vehicle_type', 'callsign'])
        [["resource_use_duration"]]
        .sum()
        )

utilisation_df_per_run


event_type                        resource_use_duration
run_number vehicle_type callsign                       
1          car          C70                     65761.0
                        C71                     26971.0
                        C72                     27607.0
           helicopter   H70                     36383.0
                        H71                     10583.0
...                                                 ...
30         car          C70                     55911.0
                        C71                     13364.0
                        C72                     28233.0
           helicopter   H70                     40947.0
                        H71                     15675.0

[150 rows x 1 columns]

In [13]:
utilisation_df_per_run.reset_index(drop=False).merge(total_avail_minutes, on="callsign")

,run_number,vehicle_type,callsign,resource_use_duration,total_available_minutes_in_sim,callsign_group
0,1,car,C70,65761.0,416100,70
1,1,car,C71,26971.0,219000,71
2,1,car,C72,27607.0,219000,72
3,1,helicopter,H70,36383.0,416100,70
4,1,helicopter,H71,10583.0,240960,71
...,...,...,...,...,...,...
145,30,car,C70,55911.0,416100,70
146,30,car,C71,13364.0,219000,71
147,30,car,C72,28233.0,219000,72
148,30,helicopter,H70,40947.0,416100,70


In [14]:
utilisation_df_per_run_by_csg = (
        resource_use_wide.groupby(['callsign_group'])
        [["resource_use_duration"]]
        .sum()
        )
utilisation_df_per_run_by_csg = utilisation_df_per_run_by_csg.reset_index()
utilisation_df_per_run_by_csg



# import re

# total_avail_hours["callsign_group"] = total_avail_hours["callsign"].apply(lambda x: re.sub('\D', '', x))
# total_avail_hours

event_type,callsign_group,resource_use_duration
0,70.0,2989569.0
1,71.0,1080981.0
2,72.0,885313.0


In [15]:
total_avail_hours_per_csg = total_avail_minutes.groupby('callsign_group').head(1).drop(columns='callsign')
total_avail_hours_per_csg['callsign_group'] =  total_avail_hours_per_csg['callsign_group'].astype('float')
total_avail_hours_per_csg

,total_available_minutes_in_sim,callsign_group
0,416100,70.0
2,240960,71.0
4,219000,72.0


In [16]:
utilisation_df_per_run_by_csg = utilisation_df_per_run_by_csg.merge(total_avail_hours_per_csg, on="callsign_group")
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim
0,70.0,2989569.0,416100
1,71.0,1080981.0,240960
2,72.0,885313.0,219000


In [17]:
n_runs=5
utilisation_df_overall = (
        utilisation_df_per_run.groupby(['callsign', 'vehicle_type'])
        [["resource_use_duration"]]
        .sum()
        )

utilisation_df_overall["resource_use_duration"] = (
    utilisation_df_overall["resource_use_duration"] /
    n_runs
    )

utilisation_df_overall.reset_index()

event_type,callsign,vehicle_type,resource_use_duration
0,C70,car,372448.6
1,C71,car,145795.0
2,C72,car,177062.6
3,H70,helicopter,225465.2
4,H71,helicopter,70401.2


In [18]:
n_runs=5

In [19]:
total_avail_minutes['total_available_minutes_in_all_runs'] = total_avail_minutes['total_available_minutes_in_sim'] * n_runs

In [20]:
total_avail_minutes

,callsign,total_available_minutes_in_sim,callsign_group,total_available_minutes_in_all_runs
0,H70,416100,70,2080500
1,C70,416100,70,2080500
2,H71,240960,71,1204800
3,C71,219000,71,1095000
4,C72,219000,72,1095000


In [21]:
resource_use_wide, utilisation_df_overall, utilisation_df_per_run, utilisation_df_per_run_by_csg = _utilisation_result_calculation.make_utilisation_model_dataframe()
resource_use_wide

c:\DAAT\DAA_DES\app\_vehicle_calculation.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_day[callsign] = max(0, first_day[callsign] - warm_up_end.hour)
c:\DAAT\DAA_DES\app\_vehicle_calculation.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_day[callsign] = min(last_day[callsign], sim_end.hour)


event_type,P_ID,run_number,callsign_group,vehicle_type,callsign,resource_use,resource_use_end,resource_use_duration
0,1,1,71.0,car,C71,2025-02-14 12:00:00,2025-02-14 14:58:00,178.0
1,1,2,70.0,car,C70,2025-02-14 09:00:00,2025-02-14 10:12:00,72.0
2,1,3,71.0,car,C71,2025-02-14 10:00:00,2025-02-14 11:06:00,66.0
3,1,4,70.0,car,C70,2025-02-14 10:00:00,2025-02-14 11:27:00,87.0
4,1,5,70.0,car,C70,2025-02-14 22:00:00,2025-02-14 23:03:00,63.0
...,...,...,...,...,...,...,...,...
45140,1763,2,72.0,car,C72,2026-02-12 10:00:00,2026-02-12 10:44:00,44.0
45141,1764,2,71.0,helicopter,H71,2026-02-12 15:00:00,2026-02-12 15:41:00,41.0
45142,1765,2,70.0,helicopter,H70,2026-02-13 01:00:00,2026-02-13 02:03:00,63.0
45143,1766,2,71.0,helicopter,H71,2026-02-13 14:00:00,2026-02-13 14:43:00,43.0


In [22]:
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim,perc_time_in_use,PRINT_perc
0,70.0,99676.300000,416100,0.239549,24.0%
1,71.0,37216.700000,240960,0.154452,15.4%
2,72.0,29878.433333,219000,0.136431,13.6%


In [23]:
utilisation_df_overall

,callsign,vehicle_type,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,C70,car,62098.766667,416100,70,0.149240,14.9%
1,C71,car,25483.166667,219000,71,0.116361,11.6%
2,C72,car,29878.433333,219000,72,0.136431,13.6%
3,H70,helicopter,37577.533333,416100,70,0.090309,9.0%
4,H71,helicopter,11733.533333,240960,71,0.048695,4.9%


In [24]:
utilisation_df_per_run

,run_number,vehicle_type,callsign,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,1,car,C70,65761.0,416100,70,0.158041,15.8%
1,1,car,C71,26971.0,219000,71,0.123155,12.3%
2,1,car,C72,27607.0,219000,72,0.126059,12.6%
3,1,helicopter,H70,36383.0,416100,70,0.087438,8.7%
4,1,helicopter,H71,10583.0,240960,71,0.043920,4.4%
...,...,...,...,...,...,...,...,...
145,30,car,C70,55911.0,416100,70,0.134369,13.4%
146,30,car,C71,13364.0,219000,71,0.061023,6.1%
147,30,car,C72,28233.0,219000,72,0.128918,12.9%
148,30,helicopter,H70,40947.0,416100,70,0.098407,9.8%


In [25]:
import plotly.express as px

px.box(utilisation_df_per_run.reset_index(),
       x="perc_time_in_use",
       y="callsign",
       color="vehicle_type")

In [26]:
px.bar(utilisation_df_overall.reset_index(),
       y="perc_time_in_use",
       x="callsign",
       color="vehicle_type")

In [27]:
_utilisation_result_calculation.make_SIMULATION_utilisation_summary_plot(utilisation_df_overall=utilisation_df_overall)

In [28]:
_utilisation_result_calculation.make_SIMULATION_utilisation_variation_plot(utilisation_df_per_run=utilisation_df_per_run)

In [29]:
utilisation_df_per_run_by_csg

,callsign_group,resource_use_duration,total_available_minutes_in_sim,perc_time_in_use,PRINT_perc
0,70.0,99676.300000,416100,0.239549,24.0%
1,71.0,37216.700000,240960,0.154452,15.4%
2,72.0,29878.433333,219000,0.136431,13.6%


## Available time calculation

In [30]:
import pandas as pd
hems_rota = pd.read_csv("../data/hems_rota_used.csv")
hems_rota[['callsign', 'summer_start', 'winter_start', 'summer_end', 'winter_end']]

,callsign,summer_start,winter_start,summer_end,winter_end
0,H70,7,7,2,2
1,CC70,7,7,2,2
2,H71,7,7,19,17
3,CC71,9,7,19,17
4,CC72,8,8,18,18


In [31]:
params = pd.read_csv("../data/run_params_used.csv")
params

,parameter,value
0,sim_duration,525600.0
1,warm_up_duration,0.0
2,sim_start_date,2025-02-14 08:00:00
3,sim_end_date,2026-02-14 08:00:00
4,warm_up_end_date,2025-02-14 08:00:00
5,amb_data,False
6,model_exec_time,2025-02-14 11:51:12.838309
7,summer_start_date,2025-04-01
8,winter_start_date,2025-10-01


In [32]:
import _processing_functions
from datetime import datetime

In [33]:
warm_up_end = _processing_functions.get_param("warm_up_end_date", params)
warm_up_end = datetime.strptime(warm_up_end, "%Y-%m-%d %H:%M:%S")
sim_end = _processing_functions.get_param("sim_end_date", params)
sim_end = datetime.strptime(sim_end, "%Y-%m-%d %H:%M:%S")


date_range = pd.date_range(start=warm_up_end.date(),
                           end=sim_end.date(),
                           freq='D')
df = pd.DataFrame({'date': date_range})

summer_start = datetime.strptime(_processing_functions.get_param("summer_start_date", params), "%Y-%m-%d")
winter_start = datetime.strptime(_processing_functions.get_param("winter_start_date", params), "%Y-%m-%d")

summer_start_month_day = pd.to_datetime(summer_start).strftime('%m-%d')
winter_start_month_day = pd.to_datetime(winter_start).strftime('%m-%d')

def is_summer(date):
        year = date.year
        summer_start_date = pd.to_datetime(f"{year}-{summer_start_month_day}")
        winter_start_date = pd.to_datetime(f"{year}-{winter_start_month_day}")
        return summer_start_date <= date < winter_start_date

df['is_summer'] = df['date'].apply(is_summer)

# Fill with a blank column for each callsign
for callsign in hems_rota.callsign:
    df[callsign] = 0

df

,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,0,0,0,0,0
1,2025-02-15,False,0,0,0,0,0
2,2025-02-16,False,0,0,0,0,0
3,2025-02-17,False,0,0,0,0,0
4,2025-02-18,False,0,0,0,0,0
...,...,...,...,...,...,...,...
361,2026-02-10,False,0,0,0,0,0
362,2026-02-11,False,0,0,0,0,0
363,2026-02-12,False,0,0,0,0,0
364,2026-02-13,False,0,0,0,0,0


In [34]:
def update_availability(df, rota):
    df = df.copy()

    for _, row in rota.iterrows():
        callsign = row['callsign']
        summer_start, winter_start = row['summer_start'], row['winter_start']
        summer_end, winter_end = row['summer_end'], row['winter_end']

        for i in range(len(df)):
            is_summer = df.at[i, 'is_summer']
            start_time = summer_start if is_summer else winter_start
            end_time = summer_end if is_summer else winter_end

            # Handle end time past midnight
            if end_time < start_time:
                duration = (24 - start_time) + end_time
                df.at[i, callsign] = duration

                if i + 1 < len(df):  # Add extra hours to next day
                    df.at[i + 1, callsign] = df.at[i + 1, callsign] + end_time
            else:
                df.at[i, callsign] = end_time - start_time

    # Adjust for simulation start time
    first_day = df.iloc[0]
    for callsign in rota['callsign']:
        if first_day[callsign] > 0:
            first_day[callsign] = max(0, first_day[callsign] - warm_up_end.hour)
    df.iloc[0] = first_day

    # Adjust for simulation end time
    last_day = df.iloc[-1]
    for callsign in rota['callsign']:
        if last_day[callsign] > 0:
            last_day[callsign] = min(last_day[callsign], sim_end.hour)
    df.iloc[-1] = last_day

    return df

update_availability(df, hems_rota)

C:\Users\Sammi\AppData\Local\Temp\ipykernel_29692\87864462.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sammi\AppData\Local\Temp\ipykernel_29692\87864462.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [35]:
df_avail = update_availability(df, hems_rota)
df_avail

C:\Users\Sammi\AppData\Local\Temp\ipykernel_29692\87864462.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sammi\AppData\Local\Temp\ipykernel_29692\87864462.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,is_summer,H70,CC70,H71,CC71,CC72
0,2025-02-14,False,11,11,2,2,2
1,2025-02-15,False,19,19,10,10,10
2,2025-02-16,False,19,19,10,10,10
3,2025-02-17,False,19,19,10,10,10
4,2025-02-18,False,19,19,10,10,10
...,...,...,...,...,...,...,...
361,2026-02-10,False,19,19,10,10,10
362,2026-02-11,False,19,19,10,10,10
363,2026-02-12,False,19,19,10,10,10
364,2026-02-13,False,19,19,10,10,10


In [36]:
avail_hours = df_avail.drop(columns=['is_summer']).sum(axis=0, numeric_only=True)

## Headline figure calculation

In [37]:
utilisation_df_overall

,callsign,vehicle_type,resource_use_duration,total_available_minutes_in_sim,callsign_group,perc_time_in_use,PRINT_perc
0,C70,car,62098.766667,416100,70,0.149240,14.9%
1,C71,car,25483.166667,219000,71,0.116361,11.6%
2,C72,car,29878.433333,219000,72,0.136431,13.6%
3,H70,helicopter,37577.533333,416100,70,0.090309,9.0%
4,H71,helicopter,11733.533333,240960,71,0.048695,4.9%


In [38]:
vehicle_type = "helicopter"
utilisation_df_overall[utilisation_df_overall["vehicle_type"] == vehicle_type].mean(numeric_only=True)['perc_time_in_use']

0.0695019212782771

In [39]:
utilisation_df_overall.groupby('callsign_group').count()[['callsign']]

,callsign
callsign_group,
70,2
71,2
72,1


In [40]:
vehicles_per_callsign_group = utilisation_df_overall.groupby('callsign_group').count()[['callsign']]


car_only = vehicles_per_callsign_group[vehicles_per_callsign_group['callsign'] == 1]
car_only


,callsign
callsign_group,
72,1


In [41]:
utilisation_df_overall[utilisation_df_overall["callsign_group"].isin(car_only.reset_index().callsign_group.values)].mean(numeric_only=True)['perc_time_in_use']


0.13643120243531204

In [42]:
# backupcar_only = vehicles_per_callsign_group[vehicles_per_callsign_group['callsign'] == 2]
# utilisation_df_overall = utilisation_df_overall[utilisation_df_overall["vehicle_type"] == "car"]
# utilisation_df_overall[utilisation_df_overall["callsign_group"].isin(backupcar_only.reset_index().callsign_group.values)].mean(numeric_only=True)['perc_time_in_use']

In [43]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("helicopter", utilisation_df_overall)

0.0695019212782771

In [44]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("helicopter backup car", utilisation_df_overall)

0.1328007490186654

In [45]:
_utilisation_result_calculation.make_SIMULATION_utilisation_headline_figure("solo car", utilisation_df_overall)

0.13643120243531204